In [4]:
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW, get_scheduler
from torch.utils.data import DataLoader, TensorDataset
import torch
from tqdm.auto import tqdm

C:\Users\m7mds\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [1]:
import pandas as pd

# Load the data
file_path = 'processed_lexioms.xlsx'
df = pd.read_excel(file_path)


In [2]:
# Mapping polarity to numerical labels
label_mapping = {'compound_pos': 1,'positive': 1, 'compound_neg': 0, 'negative': 0 }  # Adjust based on your dataset
df['label'] = df['Polarity'].map(label_mapping)

# Filter out rows with missing labels
df = df.dropna(subset=['label'])


In [5]:
from transformers import AutoTokenizer

model_name = "aubmindlab/bert-base-arabertv02"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_data(df, tokenizer):
    return tokenizer(
        list(df['Term']),
        padding=True,
        truncation=True,
        max_length=512,
        return_tensors='pt'
    )

encodings = tokenize_data(df, tokenizer)
labels = torch.tensor(df['label'].values)


In [6]:
dataset = TensorDataset(encodings['input_ids'], encodings['attention_mask'], labels)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)


In [8]:
import numpy as np

def train(model, train_loader, optimizer, lr_scheduler, device):
    model.train()
    progress_bar = tqdm(train_loader, desc="Training")
    for batch in progress_bar:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

        progress_bar.set_postfix(loss=loss.item())

def evaluate(model, val_loader, device):
    model.eval()
    progress_bar = tqdm(val_loader, desc="Evaluating")
    all_labels = []
    all_preds = []
    for batch in progress_bar:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        with torch.no_grad():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=-1)

        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(preds.cpu().numpy())

    accuracy = np.mean(np.array(all_labels) == np.array(all_preds))
    print(f"Validation Accuracy: {accuracy:.4f}")
    return accuracy, all_labels, all_preds

# Initialize model and tokenizer
model_name = "aubmindlab/bert-base-arabertv02"
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
tokenizer = AutoTokenizer.from_pretrained(model_name)

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

# Prepare optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=5e-5)
num_epochs = 1
num_training_steps = num_epochs * len(train_loader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

# Training and evaluation loop
for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")
    train(model, train_loader, optimizer, lr_scheduler, device)
    accuracy, all_labels, all_preds = evaluate(model, val_loader, device)

# Save the fine-tuned model
model.save_pretrained("fine_tuned_arabert")
tokenizer.save_pretrained("fine_tuned_arabert")

# Save evaluation results
results_df = pd.DataFrame({
    'label': all_labels,
    'prediction': all_preds
})
results_df.to_excel('evaluation_results.xlsx', index=False)
print("Fine-tuned model and evaluation results saved.")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at aubmindlab/bert-base-arabertv02 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/1


Training:   0%|          | 0/298 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/75 [00:00<?, ?it/s]

Validation Accuracy: 0.9134
Fine-tuned model and evaluation results saved.
